In [1]:
# General use
import re
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

# Notebook behavior
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
seed = 19  # Replicability

In [2]:
# Load data
df = pd.read_parquet("cleaned_events.parquet").fillna(np.nan)
df.shape
df.head()

(883292, 13)

,QuadClass,SQLDATE,Actor1CountryCode,Actor2CountryCode,GoldsteinScore,CAMEOEvent,EventDateTime,Actor1Code,Actor1Name,Actor1Geo_FullName,Actor2Code,Actor2Name,Actor2Geo_FullName
GLOBALEVENTID,,,,,,,,,,,,,
1078326026,1,20230101,NaN,NaN,2.8,043,2023-01-01,NaN,NaN,NaN,COP,POLICE OFFICER,NaN
1078326027,4,20230101,NaN,NaN,-5.0,173,2023-01-01,NaN,NaN,NaN,COP,DETECTIVE,NaN
1078326030,4,20230101,NaN,NaN,-10.0,190,2023-01-01,NaN,NaN,NaN,COP,DETECTIVE,NaN
1078326080,2,20230101,NaN,RUS,10.0,0874,2023-01-01,NaN,NaN,NaN,RUS,MOSCOW,"Moscow, Moskva, Russia"
1078326081,2,20230101,NaN,RUS,10.0,0874,2023-01-01,NaN,NaN,NaN,RUS,MOSCOW,"Monroe County, Pennsylvania, United States"


In [3]:
# Limit to 30 days
# df = df[df["SQLDATE"]>=20240101]
lim = df["SQLDATE"].drop_duplicates().sort_values(ascending=False)[30]
df = df[df["SQLDATE"]>=lim]
df = df.drop(columns=["SQLDATE"], axis=1)
df.shape

(46061, 12)

In [4]:
# Check dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46061 entries, 1157818675 to 1164123333
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   QuadClass           46061 non-null  object        
 1   Actor1CountryCode   29311 non-null  object        
 2   Actor2CountryCode   23351 non-null  object        
 3   GoldsteinScore      46061 non-null  float64       
 4   CAMEOEvent          46061 non-null  object        
 5   EventDateTime       46061 non-null  datetime64[ns]
 6   Actor1Code          42539 non-null  object        
 7   Actor1Name          42539 non-null  object        
 8   Actor1Geo_FullName  41149 non-null  object        
 9   Actor2Code          33887 non-null  object        
 10  Actor2Name          33887 non-null  object        
 11  Actor2Geo_FullName  33096 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 4.6+ MB


In [5]:
# Describe
df.describe()
df.describe(include="O")

,GoldsteinScore,EventDateTime
count,46061.000000,46061
mean,0.588589,2024-03-02 16:49:39.607477248
min,-10.000000,2024-02-15 00:00:00
25%,-2.000000,2024-02-25 00:00:00
50%,1.900000,2024-03-03 10:15:00
75%,3.400000,2024-03-10 05:30:00
max,10.000000,2024-03-16 11:30:00
std,4.555038,NaN


,QuadClass,Actor1CountryCode,Actor2CountryCode,CAMEOEvent,Actor1Code,Actor1Name,Actor1Geo_FullName,Actor2Code,Actor2Name,Actor2Geo_FullName
count,46061,29311,23351,46061,42539,42539,41149,33887,33887,33096
unique,4,167,159,185,821,1343,1519,784,1218,1395
top,1,PHL,PHL,010,PHL,PHILIPPINE,"Manila, Manila, Philippines",PHL,PHILIPPINE,"Manila, Manila, Philippines"
freq,29236,9769,6980,4073,8354,7062,3747,5842,5238,2959


# 1. Data prep prerequisites

In [6]:
# Unique IDs
df = df.reset_index()

In [7]:
# Cleaning- ensure values are normalized
# Capitalize all str cols
str_cols = df.select_dtypes(include="O")
for str_col in str_cols:
    df[str_col] = df[str_col].str.strip().str.upper().str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")

In [8]:
# Null-handling
# Note: Splink treats null values differently from empty strings, so using true nulls guarantees proper matching across datasets.
df.isna().mean()

GLOBALEVENTID         0.000000
QuadClass             0.000000
Actor1CountryCode     0.363648
Actor2CountryCode     0.493042
GoldsteinScore        0.000000
CAMEOEvent            0.000000
EventDateTime         0.000000
Actor1Code            0.076464
Actor1Name            0.076464
Actor1Geo_FullName    0.106641
Actor2Code            0.264302
Actor2Name            0.264302
Actor2Geo_FullName    0.281475
dtype: float64

# 2. Exploratory analysis

In [9]:
# Initialise the linker
from splink.duckdb.linker import DuckDBLinker
settings = {"link_type": "dedupe_only", "unique_id_column_name":"GLOBALEVENTID"}
linker = DuckDBLinker(df, settings)

In [10]:
# Check for sparsity
import altair as alt
linker.missingness_chart()

alt.LayerChart(...)

In [11]:
# Profile
linker.profile_columns(top_n=10, bottom_n=10)

alt.VConcatChart(...)

# 3. Blocking

In [12]:
from splink.duckdb.blocking_rule_library import block_on
blocking_rules = [
    block_on(["QuadClass", "Actor1CountryCode", "Actor2CountryCode"]), 
    block_on(["Actor1Name", "Actor2Name"]), 
]

{blocking_rule.blocking_rule_sql:linker.count_num_comparisons_from_blocking_rule(blocking_rule) for blocking_rule in blocking_rules}

{'(l."QuadClass" = r."QuadClass") AND (l."Actor1CountryCode" = r."Actor1CountryCode") AND (l."Actor2CountryCode" = r."Actor2CountryCode")': 745119,
 '(l."Actor1Name" = r."Actor1Name") AND (l."Actor2Name" = r."Actor2Name")': 610072}

In [13]:
# How much of initial comparison space will now be compared
n_base_comparisons = (df.shape[0]**2) - df.shape[0]  # Without blocking rule
n_comparisons = np.sum([linker.count_num_comparisons_from_blocking_rule(blocking_rule) for blocking_rule in blocking_rules])
np.round(n_comparisons / n_base_comparisons, 4)

0.0006

In [14]:
# Update settings
settings["blocking_rules_to_generate_predictions"] = blocking_rules

# 4. Estimating Model Parameters

In [15]:
# Not levenshtein because we don't expect mispellings
from splink.duckdb import comparison_library as cl
from splink.duckdb import comparison_template_library as ctl
comparisons = [
    # ctl.date_comparison("EventDateTime", datediff_thresholds=[7], datediff_metrics=["day"])
    cl.datediff_at_thresholds("EventDateTime", date_metrics=["day"], date_thresholds=[7]), 
    cl.jaro_winkler_at_thresholds("Actor1Geo_FullName", 0.8, term_frequency_adjustments=True), 
    cl.jaro_winkler_at_thresholds("Actor2Geo_FullName", 0.8, term_frequency_adjustments=True), 
    ctl.name_comparison("Actor1Name"), 
    ctl.name_comparison("Actor2Name"), 
    cl.levenshtein_at_thresholds("Actor1Code", 3, term_frequency_adjustments=True), 
    cl.levenshtein_at_thresholds("Actor2Code", 3, term_frequency_adjustments=True), 
    cl.exact_match("CAMEOEvent", term_frequency_adjustments=True), 
]
settings["comparisons"] = comparisons

In [16]:
# Update model-based settings
settings["retain_matching_columns"] = True
settings["retain_intermediate_calculation_columns"] = True
settings["max_iterations"] = 20
settings["em_convergence"] = 0.001

In [17]:
# Instantiate new linker based on updated settings
linker = DuckDBLinker("df", settings)
linker.estimate_probability_two_random_records_match(
    blocking_rules,
    recall=0.8,
)
linker.estimate_u_using_random_sampling(max_pairs=1e9)

# Note: IndexError is a bug (https://github.com/moj-analytical-services/splink/issues/2076#issuecomment-2007755672)
# Sol'n: cona install sqlglot=22.5.0
feature = "GoldsteinScore"
# linker.estimate_parameters_using_expectation_maximisation(
#     f"abs(l.{feature} - r.{feature}) <= {np.round(df[feature].std()/2, 4)}", 
# )
linker.estimate_parameters_using_expectation_maximisation(
    block_on(["Actor1Name", "QuadClass"]), 
    estimate_without_term_frequencies=False,
)

Probability two random records match is estimated to be  0.00131.
This means that amongst all possible pairwise record comparisons, one in 761.74 are expected to match.  With 1,060,784,830 total possible comparisons, we expect a total of around 1,392,582.50 matching pairs
----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - EventDateTime (no m values are trained).
    - Actor1Geo_FullName (no m values are trained).
    - Actor2Geo_FullName (no m values are trained).
    - Actor1Name (no m values are trained).
    - Actor2Name (no m values are trained).
    - Actor1Code (no m values are trained).
    - Actor2Code (no m values are trained).
    - CAMEOEvent (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."Actor1Name" = r."Actor1Name") AND (l."QuadClass" = r."QuadClass")

Parameter estimates will be made for the following comparison(s):
    - EventDateTime
    - Actor1Geo_FullName
    - Actor2Geo_FullName
    - Actor2Name
    - Actor1Code
    - Actor2Code
    - CAMEOEvent

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - Actor1Name


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

SplinkException: Error executing the following sql for table `__splink__m_u_counts`(__splink__m_u_counts_470373934):
CREATE TABLE __splink__m_u_counts_470373934 AS
(
  WITH __splink__df_comparison_vectors AS (
    SELECT
      *
    FROM __splink__df_comparison_vectors_10fec52fb
  ), __splink__df_match_weight_parts AS (
    SELECT
      "GLOBALEVENTID_l",
      "GLOBALEVENTID_r",
      gamma_EventDateTime,
      CASE
        WHEN gamma_EventDateTime = -1
        THEN CAST(1.0 AS DOUBLE)
        WHEN gamma_EventDateTime = 2
        THEN CAST(937.0115532372255 AS DOUBLE)
        WHEN gamma_EventDateTime = 1
        THEN CAST(0.05582759229774678 AS DOUBLE)
        WHEN gamma_EventDateTime = 0
        THEN CAST(0.04535732418739619 AS DOUBLE)
      END AS bf_EventDateTime,
      gamma_Actor1Geo_FullName,
      CASE
        WHEN gamma_Actor1Geo_FullName = -1
        THEN CAST(1.0 AS DOUBLE)
        WHEN gamma_Actor1Geo_FullName = 2
        THEN CAST(39.58022767144771 AS DOUBLE)
        WHEN 

# 5. Predicting results

In [ ]:
# Predict
threshold = 0.95
pred = linker.predict(threshold_match_probability=threshold)
pred_df = pred.as_pandas_dataframe()

clusters = linker.cluster_pairwise_predictions_at_threshold(
    pred, threshold_match_probability=threshold
).as_pandas_dataframe()

n_pairwise_comparisons = (
    linker.count_num_comparisons_from_blocking_rules_for_prediction(
        pred
    ).as_pandas_dataframe(limit=None)
).count_of_edges.sum()
n_base_comparisons, n_comparisons, n_pairwise_comparisons

In [ ]:
# Check model parameters
linker.m_u_parameters_chart()

Ref: https://www.robinlinacre.com/fast_deduplication/

clusters[clusters["cluster_id"]==clusters["cluster_id"].value_counts().index[0]]

import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl

settings = {
    "link_type": "dedupe_only",
    "unique_id_column_name":"GLOBALEVENTID", 
    "blocking_rules_to_generate_predictions": [
        block_on(["QuadClass", "Actor1CountryCode", "Actor2CountryCode"]), 
    ],
    "comparisons": [
        ctl.name_comparison("Actor1Name", term_frequency_adjustments=True),
        ctl.name_comparison("Actor2Name", term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("Actor1Geo_FullName", [1, 2]),
        cl.datediff_at_thresholds("EventDateTime", ), 
        cl.jaro_at_thresholds("full_name", [0.9, 0.7], term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("dob", [1, 2]),
        cl.levenshtein_at_thresholds("postcode_fake", 2),
        cl.jaro_winkler_at_thresholds("birth_place", 0.9, term_frequency_adjustments=True),
        cl.exact_match("occupation",  term_frequency_adjustments=True),
    ],           

    'comparison_levels': [
    {
        'sql_condition': '"date_of_birth_l" IS NULL OR "date_of_birth_r" IS NULL',
        'label_for_charts': 'Null',
        'is_null_level': True
    },
    "comparisons": [
        ctl.date_comparison("EventDateTime", cast_strings_to_date=True, )
        ctl.name_comparison("first_name"),
        ctl.name_comparison("surname"),
        ctl.date_comparison("dob", cast_strings_to_date=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        ctl.email_comparison("email", include_username_fuzzy_level=False),
    ],"retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}

linker = DuckDBLinker(df, settings)